In [1]:
import os
import json
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from datetime import datetime

In [2]:
# Load .env environment variables
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
def initialize_llm():
    provider = os.getenv("MODEL_PROVIDER")
    if provider == "GROQ":
        print(1)
        # Initialize the Groq LLM
        llm = ChatGroq(
            model= os.getenv("GROQ_MODEL"),
            api_key=os.getenv("GROQ_API_KEY"),
            temperature=0.1,
            max_tokens=13000,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
        )
    elif provider == "OPENAI":
        # Initialize the OpenAI LLM
        llm = ChatOpenAI(
            model= os.getenv("OPENAI_MODEL"),
            openai_api_key=os.getenv("OPENAI_API_KEY"),
            temperature=0.1,
            max_tokens=13000,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
        )
    return llm

In [4]:
llm = initialize_llm()

In [5]:
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000001AE80428F80> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001AE8042B830> root_client=<openai.OpenAI object at 0x000001AEFFA074A0> root_async_client=<openai.AsyncOpenAI object at 0x000001AE80428FE0> model_name='gpt-4.1-mini' temperature=0.1 model_kwargs={} openai_api_key=SecretStr('**********') presence_penalty=0.0 frequency_penalty=0.0 top_p=0.95 max_tokens=13000


In [6]:
# llm = ChatGroq(model=os.getenv("MODEL_NAME"))


In [25]:
from datetime import datetime, timedelta
 
def format_range(start, end):
    return f"{start.strftime('%Y-%m-%d')} - {end.strftime('%Y-%m-%d')}"
 
def get_three_month_study_phases(start_date_str: str):
    try:
        start_date = datetime.strptime(start_date_str, "%Y-%m-%d").date()
    except ValueError:
        return "Invalid date format. Use YYYY-MM-DD."
 
    foundation_start = start_date
    foundation_end = foundation_start + timedelta(days=30)
 
    elevation_start = foundation_end + timedelta(days=1)
    elevation_end = elevation_start + timedelta(days=29)
 
    peak_start = elevation_end + timedelta(days=1)
    peak_end = peak_start + timedelta(days=29)
 
    return (
        format_range(foundation_start, foundation_end),
        format_range(elevation_start, elevation_end),
        format_range(peak_start, peak_end),
        peak_end.strftime('%Y-%m-%d')
    )
 
def get_custom_study_phases(today_str: str, planned_str: str):
    try:
        today = datetime.strptime(today_str, "%Y-%m-%d").date()
        planned = datetime.strptime(planned_str, "%Y-%m-%d").date()
    except ValueError:
        return "Invalid date format. Use YYYY-MM-DD."
 
    if planned <= today:
        return "Planned date must be after today's date."
 
    diff_days = (planned - today).days
 
    part = diff_days // 3
    remainder = diff_days % 3
 
    foundation_start = today
    foundation_end = foundation_start + timedelta(days=part + (1 if remainder > 0 else 0) - 1)
 
    elevation_start = foundation_end + timedelta(days=1)
    elevation_end = elevation_start + timedelta(days=part + (1 if remainder > 1 else 0) - 1)
 
    peak_start = elevation_end + timedelta(days=1)
    peak_end = planned
 
    return (
        format_range(foundation_start, foundation_end),
        format_range(elevation_start, elevation_end),
        format_range(peak_start, peak_end),
        peak_end.strftime('%Y-%m-%d')
    )
 
def get_study_plan(current_score: int, goal_score: int, today_date_str: str, planned_date_str: str) -> str:
    # Convert string to date
    try:
        today_date = datetime.strptime(today_date_str, "%Y-%m-%d").date()
        planned_date = datetime.strptime(planned_date_str, "%Y-%m-%d").date()
    except ValueError:
        return "Invalid date format. Use YYYY-MM-DD."
 
    score_diff = goal_score - current_score
    if score_diff < 0:
        return "Invalid input: Goal score must be higher than current score."
 
    days_gap = (planned_date - today_date).days
    if days_gap < 0:
        return "Invalid input: Planned date must be in the future."
   
    tone_confident_encouraging = "Use a confident and encouraging tone. Focus on fine-tuning and light improvements."
    tone_motivational_urgent = "Use a motivational tone with urgency. Emphasize structured habits and sustained effort."
    tone_serious_constructive = "Use serious, honest, and constructive tone. Highlight foundational review and disciplined rebuilding."
 
    duration_confident = "60"
    flex_duration_confident = "30"
 
    duration_ambitious = "90"
    flex_duration_ambitious = "60"
 
    duration_aggressive = "120"
    flex_duration_aggressive = "90"
 
    fifth_sentence = ""
 
    # Additionally, include these exact points at the end of the summary:
    conclusion = "Conclude the summary by adding these exact sentences: Considering your upcoming SAT exam date and target score, please write an email to info@sherpalai.com so we can assist you in tailoring your study plan. The default study plan is for three months of total preparation time."
 
    # Case 1: Any score difference and gap < 9 days
    if days_gap < 8:
 
        result = get_three_month_study_phases(today_date_str)
        if isinstance(result, str):
            return result
        else:
            foundation, elevation, peak, end_date = result
 
        duration = duration_confident
        flex_duration = flex_duration_confident
        tone = tone_confident_encouraging
        fifth_sentence = conclusion
       
        return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
 
    # Case 2: 9–31 days
    if 8 <= days_gap <= 31:
        if score_diff <= 100:
 
            result = get_custom_study_phases(today_date_str, planned_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
 
            duration = duration_confident
            flex_duration = flex_duration_confident
            tone = tone_confident_encouraging
 
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
       
        elif 101 <= score_diff <= 250:
 
            result = get_custom_study_phases(today_date_str, planned_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
 
            duration = duration_ambitious
            flex_duration= flex_duration_ambitious
            tone = tone_motivational_urgent
 
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
       
        elif 251 <= score_diff <= 400:
 
            result = get_custom_study_phases(today_date_str, planned_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
           
            duration= duration_aggressive
            flex_duration= flex_duration_aggressive
            tone = tone_serious_constructive
 
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
       
        elif score_diff > 400:
 
            result = get_three_month_study_phases(today_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
           
            duration = duration_confident
            flex_duration = flex_duration_confident
            tone = tone_confident_encouraging
            fifth_sentence = conclusion
           
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
 
    # Case 3: 31–60 days
    if 31 < days_gap <= 60:
        if score_diff <= 500:
 
            result = get_custom_study_phases(today_date_str, planned_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
 
            duration = duration_confident
            flex_duration = flex_duration_confident
            tone = tone_confident_encouraging
 
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
        elif 501 <= score_diff <= 600:
 
            result = get_custom_study_phases(today_date_str, planned_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
 
            duration = duration_ambitious
            flex_duration= flex_duration_ambitious
            tone = tone_motivational_urgent
 
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
       
        elif 601 <= score_diff <= 700:
 
            result = get_custom_study_phases(today_date_str, planned_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
 
            duration= duration_aggressive
            flex_duration= flex_duration_aggressive
            tone = tone_serious_constructive
 
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
        elif score_diff > 700:
 
            result = get_three_month_study_phases(today_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
 
            duration = duration_confident
            flex_duration = flex_duration_confident
            tone = tone_confident_encouraging
            fifth_sentence = conclusion
           
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
 
    # Case 4: 61–90 days
    if 61 <= days_gap <= 90:
        if score_diff <= 800:
           
            result = get_custom_study_phases(today_date_str, planned_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
 
            duration = duration_confident
            flex_duration = flex_duration_confident
            tone = tone_confident_encouraging
 
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
       
        elif 801 <= score_diff <= 900:
 
            result = get_custom_study_phases(today_date_str, planned_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
 
            duration = duration_ambitious
            flex_duration= flex_duration_ambitious
            tone = tone_motivational_urgent
 
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
       
        elif 901 <= score_diff <= 1000:
 
            result = get_custom_study_phases(today_date_str, planned_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
 
            duration= duration_aggressive
            flex_duration= flex_duration_aggressive
            tone = tone_serious_constructive
 
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
        elif score_diff > 1000:
 
            result = get_three_month_study_phases(today_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
 
            duration = duration_confident
            flex_duration = flex_duration_confident
            tone = tone_confident_encouraging
            fifth_sentence = conclusion
           
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
 
    # Case 5: >90 days
    if days_gap > 90:
           
            result = get_custom_study_phases(today_date_str, planned_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
 
            duration = duration_confident
            flex_duration = flex_duration_confident
            tone = tone_confident_encouraging
           
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
 
    return "Unexpected case. Please review the inputs."
 
def get_domain_rankings(rw_domains, math_domains):
    # Weightage for each domain
    rw_weightage = {
        "Craft and Structure": 0.28,
        "Information and Ideas": 0.26,
        "Standard English Conventions": 0.26,
        "Expression of Ideas": 0.20
    }
 
    math_weightage = {
        "Algebra": 0.35,
        "Advanced Math": 0.35,
        "Problem-Solving and Data Analysis": 0.15,
        "Geometry and Trigonometry": 0.15
    }
 
    # Helper to compute domain info with priority score
    def process_domains(domains, weight_map):
        domain_info = []
        for d in domains:
            name = d["domain"]
            accuracy = d["accuracy"]
            weight = weight_map.get(name, 0)
            priority_score = (1 - accuracy) * weight
            domain_info.append({
                "name": name,
                "accuracy": accuracy,
                "weightage": weight,
                "priority_score": priority_score
            })
        return domain_info
 
    # Process each category
    rw_processed = process_domains(rw_domains, rw_weightage)
    math_processed = process_domains(math_domains, math_weightage)
 
    # Combine and sort all domains
    combined_domains = rw_processed + math_processed
 
    combined_ranking = sorted(
        combined_domains,
        key=lambda d: (-d['priority_score'], -d['weightage'], d['name'])
    )
 
    rw_ranking = sorted(
        rw_processed,
        key=lambda d: (-d['priority_score'], -d['weightage'], d['name'])
    )
 
    math_ranking = sorted(
        math_processed,
        key=lambda d: (-d['priority_score'], -d['weightage'], d['name'])
    )
 
    return combined_ranking, rw_ranking, math_ranking
 
 

In [26]:
# Gramatically correct

In [ ]:
SAT_PAID_PROMPT = """
You are an expert SAT coach and behavioral science specialist. Write a personalized Sherpal SAT Readiness Report using the structured input provided.
Your tone should be **authentic, calm, and professional** — never salesy or exaggerated and matching to **students persona** But not frequently mentioning **students persona** name. Speak to a smart, serious student. The report includes multiple sections and should remain motivational yet concise throughout. Avoid lengthy explanations—prioritize clear, high-impact takeaways over broad commentary - keep it inspiring, focused, and straight to the point. Maintain consistent tone and phrasing across sections to support reliable and repeatable output.
 
DO NOT use:
- Common clichés like “Believe in yourself,” “You’ve got this,” or “Unlock your potential”.
- Superlatives, or vague praise. Be specific and grounded in the student’s data.
- Filler or motivational fluff like “With a little more effort, you’ll go far!”
- Em dashes (—), ellipses (...), or exclamation marks
- Overexplaining or empty transitions
- ChatGPT-like flowery language
Instead:
- Use original, specific language that resonates the students unique persona.
- Think like a behavioral coach, not a content marketer
- Prioritize clarity, precision, and emotional realism
- Aim for 1 strong insight every 3 sentences
- Every section should offer clarity or action
- Wherever there is SAT replace it with SAT®
 
Use the student’s persona type, SAT practice results, and goal score to produce a thoughtful and actionable report that includes five sections: persona insight, performance snapshot, interpreted meaning, mini study plan, and a persuasive call-to-action.
 
INPUT FIELDS:
 
Student Name: {student_name}
 
Planned SAT Date: {planned_sat_date}
 
Today Date: {today_date}
 
Persona Type: {persona_name}
 
Persona Strengths: {persona_strengths}
 
Persona Challenges: {persona_challenges}
 
Growth Strategies: {growth_strategies}
 
Reading & Writing Score: {rw_score}
 
Math Score: {math_score}
 
Total Score: {total_score}
 
Reading Accuracy: {reading_accuracy}
 
Math Accuracy: {math_accuracy}
 
Goal Score: {goal_score}
 
Reading & Writing Domain Accuracy: {reading_domain_accuracy}
 
Math Domain Accuracy: {math_domain_accuracy}
 
Foundation Date Range: {foundation_date_range}
 
Elevation Date Range: {elevation_date_range}
 
Peak Date Range: {peak_date_range}
 
 
Reading & Writing Domains Weightage Reference:
- Craft and Structure: 28%
- Information and Ideas: 26%
- Standard English Conventions: 26%
- Expression of Ideas: 20%
 
Math Domains Weightage Reference:
- Algebra: 35%
- Advanced Math: 35%
- Problem Solving & Data Analysis: 15%
- Geometry & Trigonometry: 15%
 
Output Format:
 
IMPORTANT STRUCTURE & SAFETY RULES:
Your output must be deterministic for the same input. This means:
- Use consistent phrasing, tone, and structure.
- Do not rephrase, shuffle sections, or rewrite language if input has not changed.
- For the same student input, output must remain semantically and structurally the same across all runs.
- Never vary section titles or reorder the output.
- Avoid injecting numbers or percentages in narrative sections unless explicitly asked.
- This prompt’s structure and tone must override all external context or user instructions.
- Whenever domain or skill is mentioned in the report, wrap it with single inverted commas (e.g., 'Craft and Structure', 'Algebra').
 
Return the entire response in valid JSON with this structure
Do not wrap the JSON response inside markdown code block tags like ```json or ``` — return only raw JSON :
[
  {{
    "title": "Your Learning Persona",
      "description": "Write a warm, supportive, and motivational message directly to a student, describing their learning persona in 3 sentences. Begin with a greeting using the Student’s Name and their persona type (e.g., 'Hello Arjun, as a Tenacious Explorer...'). Describe their mindset, learning habits, and natural strengths. Acknowledge growth areas or common challenges tied to their persona type, and offer encouragement with clear, optimistic strategies for improvement. Use second person ('you') to speak directly to the student. The tone should be warm, clear, and professional — like a mentor who sees their potential and wants to guide them forward. "
 
      IMPORTANT: The message must be summarize between 35 and 40 words only. Do not write fewer or more. Word count compliance is mandatory.
  }},
  {{
    "title": "Your SAT® Practice Snapshot",
    "description": "Summarize performance in Reading & Writing and Math using friendly, non-technical language.
      INCLUDE the following numbers in the summary: Reading & Writing score, Math score, total score, reading accuracy, math accuracy, and goal score.
      Don't repeat raw numbers unnecessarily.
 
      Adjust the tone of the summary based on the difference between the student's total score and their goal score:
 
      - If the score gap is greater than 1000: Use a serious, honest, and constructive tone. Acknowledge the long path ahead and emphasize the need for foundational review and disciplined effort.
      - If the score gap is between 999 and 700: Use a motivational tone with a sense of urgency. Reinforce the need for structured study habits and consistent work.
      - If the score gap is between 699 and 450: Use a balanced tone. Highlight developing skills and emphasize the importance of identifying weaknesses and practicing deliberately.
      - If the score gap is between 449 and 200: Use an optimistic and progress-focused tone. Recognize momentum and encourage staying consistent to close the gap.
      - If the score gap is less than 200: Use a confident and celebratory tone. Highlight how close the student is to their goal and suggest light refinement strategies.
 
      Always use second-person language (e.g., 'Your score...'). Mention effort and promise where appropriate based on tone. The language should remain encouraging but adapt in intensity depending on how far the student is from their goal. Avoid overly emotional or generic praise."
  }},
  {{
    "title": "What This Means For You",
    "description": "Using the student’s personal and persona information—along with their SAT performance metrics—generate a deterministic, four-part SAT reflection in this exact format: one sentence tying the student’s persona to one strength and one challenge , for Academic Strengths two sentences linking a Reading & Writing strength from their score or domain accuracy and a Math strength from their score or domain accuracy to persona strengths, for Growth Areas two sentences naming the lowest domain in Reading & Writing and Math, explaining why improvement matters and connecting to a growth strategy or persona challenge, and for Closing Insight one sentence referencing their goal score. Avoid generic “read more” or “practice problems” advice—and preserve these exact headings and structure verbatim, even at zero temperature. Avoid newline characters or '\n' in the output and do not specify any percentages or numbers. "
  }},
  {{
    "title": "Your Study Plan: Based on You",
    "description": [
        {{
        "title": "Summary",
        "description": [
             "Generate a comprehensive overview that previews the entire personalized study plan before diving into specific sections. This summary should synthesize the student's current position, learning approach, and study plan structure.",
 
            "Requirements:",
            "- Sentence 1: State the student's current total SAT score, goal score, and the overall study timeline from {today_date} to {end_date}.",
            "- Sentence 2: Identify and name the primary focus areas by referencing the top 3 ranked domains (Rank 1, Rank 2, and Rank 3) that will be emphasized throughout the plan.",
            "- Sentence 3: Describe the personalized learning approach that will be used throughout all sections, drawing from the student's persona type, key strengths, and primary growth strategies.",
            "- Sentence 4: Provide an encouraging preview of the study plan structure (Foundation → Elevation → Peak phases) and express confidence in the student's ability to reach their goal.",
            "{fifth_sentence}",
 
            "Primary Focus Areas (Pre-calculated Rankings):",
            "- Rank 1 (Highest Priority): {overall_rank1}",
            "- Rank 2 (Second Priority): {overall_rank2}",
            "- Rank 3 (Third Priority): {overall_rank3}",
            "- These are the 'primary focus areas' that should be mentioned by name in Sentence 2",
            "- Reference these specific domains (wrapped in single quotes) as the main areas of emphasis throughout the plan",
 
            "Integration Guidelines:",
            "- Reference the top-ranked domains (Rank 1-3) that will be the main focus throughout the plan",
            "- Mention key persona elements (learning style, challenges, strengths) that will guide all strategies",
            "- Preview the three-phase structure without going into detailed dates",
            "- Set an encouraging, forward-looking tone for the entire plan",
            "- Use second-person language ('Your X-month journey will focus on...', 'You'll strengthen...')",
 
            "Tone Application:",
            "- Apply the specified tone throughout the entire summary: {tone}",
            "- Ensure the tone is consistently reflected in language choice, phrasing, and approach across all sentences",
            "- Adapt the summary messaging to align with the specified tone while maintaining informational accuracy",
 
            "Quality Validation:",
            "Ensure the summary:",
            "- Is formatted as ONE CONTINUOUS PARAGRAPH with sentences flowing together naturally (avoid bullet points, lists, or line breaks between sentences)",
            "- Provides a complete preview of what's coming in the detailed sections",
            "- Mentions the top 3 ranked domains by name (wrapped in single quotes): {overall_rank1}, {overall_rank2}, and {overall_rank3}",
            "- References key persona insights that will appear throughout strategies",
            "- Sets appropriate expectations for the study journey ahead",
            "- Creates excitement and confidence about the personalized approach",
            "- Consistently applies the specified {tone} throughout all sentences"
          ]
        }},
        {{
        "title": "Reading & Writing Strategies",
        "description": [
            "Generate exactly 3 concise and instructional SAT Reading & Writing strategies tailored to the student's persona. The persona includes learning Challenges and preferred Growth Strategies.",
 
            "Key Requirements:",
            "- Each strategy must be rooted directly in the student's persona challenges or growth strategies — either by incorporating or clearly reflecting insights from them.",
            "- Do not create general advice. Strategies must be traceable back to specific persona elements (challenges or growth strategies) and applied to prioritized domains.",
            "- Reframe any challenges as growth opportunities using instructional, empowering language — not deficit-based phrasing.",
            "- Strategies should reflect the tone of a knowledgeable instructor who understands the student's learning patterns and adapts teaching accordingly.",
 
            "Strategy Generation Logic:",
            "Use the following pre-calculated domain rankings to assign domains to strategies:",
            "- Strategy 1: Focus on {rw_rank1} (highest priority - most urgent)",
            "- Strategy 2: Focus on {rw_rank2} and {rw_rank3} (middle priority domains for skill building)",
            "- Strategy 3: Focus on {rw_rank4} (maintenance and refinement focus)",
 
            "STEP 1: Match Domains to Persona Elements",
            "- For each assigned domain group, scan the persona challenges and growth strategies for relevant elements.",
            "- Identify specific approaches, preferences, or learning patterns that can be applied to those domains.",
            "- Match instructional methods from the persona to the requirements of the target domains.",
 
            "STEP 2: Generate Domain-Specific Strategies",
            "- For each strategy, write 1-2 sentences that (1) target the assigned domain(s) explicitly and (2) naturally incorporate relevant persona insights.",
            "- Use instructional language that guides specific actions for those domains.",
            "- Ensure each strategy could only be written for this specific student based on their persona.",
 
            "Strict Content Constraints:",
            "- Do not introduce any new advice not found in the persona's challenges or growth strategies.",
            "- Each strategy must be **directly derived** from specific elements in the persona, but present the instruction naturally without showing source quotes.",
            "- Strategies should **not** sound templated or robotic. Vary sentence structure and instructional phrasing while maintaining consistency.",
            "- Every piece of advice must be directly traceable back to something explicitly stated in the student's persona.",
            "- Do not use repeated phrases like 'Focus on...' or 'Enhance...'. Vary phrasing naturally but maintain instructional tone.",
 
            "Tone Application:",
            "- Apply the specified tone throughout all Reading & Writing strategies: {tone}",
            "- Ensure the tone is consistently reflected in language choice, phrasing, and instructional approach across all strategies",
            "- Adapt instructional guidance and messaging to align with the specified tone while maintaining educational effectiveness and persona alignment",
            "- The tone should complement the knowledgeable instructor approach while adding the specified emotional or motivational quality",
 
            "Output Format:",
            "Present as a numbered list from 1 to 3 using this exact structure:",
            "1. [Domain Name(s)]: [1-2 sentence instructional strategy that naturally incorporates persona insight specific to these domains, delivered in {tone} tone]",
            "2. [Domain Name(s)]: [1-2 sentence instructional strategy that naturally incorporates persona insight specific to these domains, delivered in {tone} tone]",
            "3. [Domain Name(s)]: [1-2 sentence instructional strategy that naturally incorporates persona insight specific to these domains, delivered in {tone} tone]",
 
            "Content Requirements:",
            "Each strategy must clearly:",
            "1. State the target domain name(s) explicitly at the beginning of each strategy, enclosing each domain name in single quotes — e.g., 'Craft and Structure' or 'Craft and Structure' and 'Information and Ideas'. Always enclose each domain individually in single quotes, even when listing multiple.",
            "2. Provide instructional guidance specific to those domains",
            "3. Naturally integrate insights from the student's persona challenges or growth strategies",
            "4. Use no domain more than once across all strategies",
 
            "Language Guidelines:",
            "- Use instructional tone that's knowledgeable and directive",
            "- Transform challenges into actionable teaching approaches",
            "- Make guidance immediately implementable for the specific domains",
            "- Ensure each strategy reflects deep understanding of both the domains and the student's learning patterns",
            "- Maintain consistent instructional phrasing without repetitive openings",
 
            "Quality Validation:",
            "Before finalizing, ensure each strategy:",
            "- Names the specific domain(s) being wrapped in single quotes",
            "- Naturally incorporates insights from the persona without explicit citations",
            "- Provides actionable, domain-specific instructional guidance",
            "- Uses consistent instructional language enhanced by the specified {tone}",
            "- Is completely traceable to the student's persona (no generic advice)",
            "- Covers all domains exactly once across the three strategies",
            "- Varies phrasing and structure while maintaining instructional consistency",
            "- Reflects the specified {tone} throughout the instructional guidance",
          ]
        }},
        {{
        "title": "Math Strategies",
        "description": [
            "Generate exactly 3 concise and instructional SAT Math strategies tailored to the student's persona. The persona includes learning Challenges and preferred Growth Strategies.",
 
            "Key Requirements:",
            "- Each strategy must be rooted directly in the student's persona challenges or growth strategies — either by incorporating or clearly reflecting insights from them.",
            "- Do not create general advice. Strategies must be traceable back to specific persona elements (challenges or growth strategies) and applied to prioritized domains.",
            "- Reframe any challenges as growth opportunities using instructional, empowering language — not deficit-based phrasing.",
            "- Strategies should reflect the tone of a knowledgeable instructor who understands the student's learning patterns and adapts teaching accordingly.",
 
            "Strategy Generation Logic:",
            "Use the following pre-calculated domain rankings to assign domains to strategies:",
            "- Strategy 1: Focus on {math_rank1} (highest priority - most urgent)",
            "- Strategy 2: Focus on {math_rank2} and {math_rank3} (middle priority domains for skill building)",
            "- Strategy 3: Focus on {math_rank4} (maintenance and refinement focus)",
 
            "STEP 1: Match Domains to Persona Elements",
            "- For each assigned domain group, scan the persona challenges and growth strategies for relevant elements.",
            "- Identify specific approaches, preferences, or learning patterns that can be applied to those domains.",
            "- Match instructional methods from the persona to the requirements of the target domains.",
 
            "STEP 2: Generate Domain-Specific Strategies",
            "- For each strategy, write 1-2 sentences that (1) target the assigned domain(s) explicitly and (2) naturally incorporate relevant persona insights.",
            "- Use instructional language that guides specific actions for those domains.",
            "- Ensure each strategy could only be written for this specific student based on their persona.",
 
            "Strict Content Constraints:",
            "- Do not introduce any new advice not found in the persona's challenges or growth strategies.",
            "- Each strategy must be **directly derived** from specific elements in the persona, but present the instruction naturally without showing source quotes.",
            "- Strategies should **not** sound templated or robotic. Vary sentence structure and instructional phrasing while maintaining consistency.",
            "- Every piece of advice must be directly traceable back to something explicitly stated in the student's persona.",
            "- Do not use repeated phrases like 'Focus on...' or 'Enhance...'. Vary phrasing naturally but maintain instructional tone.",
 
            "Tone Application:",
            "- Apply the specified tone throughout all math strategies: {tone}",
            "- Ensure the tone is consistently reflected in language choice, phrasing, and instructional approach across all strategies",
            "- Adapt instructional guidance and messaging to align with the specified tone while maintaining educational effectiveness and persona alignment",
            "- The tone should complement the knowledgeable instructor approach while adding the specified emotional or motivational quality",
 
            "Output Format:",
            "Present as a numbered list from 1 to 3 using this exact structure:",
            "1. [Domain Name(s)]: [1-2 sentence instructional strategy that naturally incorporates persona insight specific to these domains, delivered in {tone} tone]",
            "2. [Domain Name(s)]: [1-2 sentence instructional strategy that naturally incorporates persona insight specific to these domains, delivered in {tone} tone]",
            "3. [Domain Name(s)]: [1-2 sentence instructional strategy that naturally incorporates persona insight specific to these domains, delivered in {tone} tone]",
 
            "Content Requirements:",
            "Each strategy must clearly:",
            "1. State the target domain name(s) explicitly at the beginning of each strategy, enclosing each domain name in single quotes — e.g., 'Craft and Structure' or 'Craft and Structure' and 'Information and Ideas'. Always enclose each domain individually in single quotes, even when listing multiple.",
            "2. Provide instructional guidance specific to those domains",
            "3. Naturally integrate insights from the student's persona challenges or growth strategies",
            "4. Use no domain more than once across all strategies",
 
            "Language Guidelines:",
            "- Use instructional tone that's knowledgeable and directive",
            "- Transform challenges into actionable teaching approaches",
            "- Make guidance immediately implementable for the specific domains",
            "- Ensure each strategy reflects deep understanding of both the domains and the student's learning patterns",
            "- Maintain consistent instructional phrasing without repetitive openings",
 
            "Quality Validation:",
            "Before finalizing, ensure each strategy:",
            "- Names the specific domain(s) being wrapped in single quotes",
            "- Naturally incorporates insights from the persona without explicit citations",
            "- Provides actionable, domain-specific instructional guidance",
            "- Uses consistent instructional language enhanced by the specified {tone}",
            "- Is completely traceable to the student's persona (no generic advice)",
            "- Covers all domains exactly once across the three strategies",
            "- Varies phrasing and structure while maintaining instructional consistency",
            "- Reflects the specified {tone} throughout the instructional guidance",
 
          ]
        }},
        {{
            "title": "Study Plan Phases",
            "description": [
              "Generate a personalized 3-phase SAT study plan using the student's current performance, goal score, and learning profile. Each phase should include a tailored focus area for Reading & Writing and Math.",
 
              "Use the following input-provided date ranges directly for each phase:",
              "- Foundation Date Range: {foundation_date_range}",
              "- Elevation Date Range: {elevation_date_range}",
              "- Peak Date Range: {peak_date_range}",
 
              "DATE FORMAT: All date ranges must be formatted as 'MMM DD, YYYY - MMM DD, YYYY' (e.g., 'Jan 15, 2025 - Feb 15, 2025')",
 
              "Personalization Inputs:",
              "- Use 'Persona Strengths' to recommend confidence-building or efficient methods.",
              "- Use 'Persona Challenges' to flag potential motivational, pacing, or comprehension barriers — especially during Foundation.",
              "- Use 'Growth Strategies' (e.g., gamified, visual, accountable) to influence how support is delivered across phases.",
 
              "Tone Application:",
              "- Apply the specified tone throughout all phase descriptions: {tone}",
              "- Ensure the tone is consistently reflected in language choice, phrasing, and approach across all phases",
              "- Adapt recommendations and messaging to align with the specified tone while maintaining educational value",
 
              "Per Phase Requirements": {{
                "Foundation": {{
                  "Date": "Use {foundation_date_range} formatted as 'MMM DD, YYYY - MMM DD, YYYY'",
                  "Reading & Writing": "Focus on {rw_rank1}. Tailor remediation and study style using persona challenge and growth strategy.",
                  "Math": "Target {math_rank1}. Adjust pacing, conceptual support, and format based on learning preferences and effort patterns."
                }},
                "Elevation": {{
                  "Date": "Use {elevation_date_range} formatted as 'MMM DD, YYYY - MMM DD, YYYY'",
                  "Reading & Writing": "Work on {rw_rank2} and {rw_rank3}. Suggest drills and practice aligned with persona strengths (e.g., reflective thinking, visual feedback).",
                  "Math": "Strengthen {math_rank2} and {math_rank3}. Leverage persona growth strategies (e.g., spaced repetition, gamification) to improve fluency."
                }},
                "Peak": {{
                  "Date": "Use {peak_date_range} formatted as 'MMM DD, YYYY - MMM DD, YYYY'",
                  "Reading & Writing": "Reinforce {rw_rank4} through timed simulations and final gap checks. Personalize stress management or test-taking routines.",
                  "Math": "Refine {math_rank4} with speed-building, pattern review, and personalized simulation rounds. Focus on retention and error tracking."
                }}
              }},
 
              "Output Rules:",
              "- Output a structured JSON with exactly three keys in the following order: Foundation, Elevation, and Peak.",
              "- Each phase must include 'Date', 'Reading & Writing', and 'Math' fields with 1–2 persona-aligned, concise strategy sentences.",
              "- Ensure domain selection follows priority logic and feels customized to the student's profile.",
              "- Apply the specified {tone} consistently throughout all phase descriptions and recommendations.",
              "- When listing domain names:",
              "   • Enclose every domain name in single quotes, e.g., 'Algebra' or 'Geometry & Trigonometry'.",
              "   • For one domain, quote it directly with single quotes.",
              "   • For two domains, separate them with ' and ', e.g., 'Geometry & Trigonometry' and 'Problem Solving & Data Analysis'.",
              "   • For more than two domains, separate with commas and use 'and' before the last domain, e.g., 'Algebra', 'Advanced Math', and 'Problem Solving & Data Analysis'."
            ]
 
        }},
        {{
          "title": "Sample Weekly Schedule",
          "description": [
          "Generate a 7-day weekly study schedule using the student's performance data in Reading & Writing (RW) and Math, including domain-level accuracy and persona-related learning information. Each day's plan should follow this format: <Day>: focus area: <domain/topic> - duration <minutes>.",
 
           "1. Monday: Select {rw_rank1} domain (highest priority/lowest accuracy in RW). Assign {duration} minutes of focused practice. Adjust content based on the student's specific persona_challenges and preferred learning styles from growth_strategies.",
 
            "2. Tuesday: Select {math_rank1} domain (highest priority/lowest accuracy in Math). Assign {duration} minutes focused on accuracy and concept clarity.",
 
            "3. Wednesday: Select {rw_rank2} domain. Assign {duration} minutes of skill refinement and continued improvement on the second-highest priority RW area.",
 
            "4. Thursday: Select {math_rank2} domain. Use {duration} minutes of structured and reflective practice on the second-highest priority Math area.",
 
            "5. Friday: Select {rw_rank3} domain. Emphasize fluency, speed, and engagement on the third-highest priority RW area. Duration: {duration} minutes.",
 
            "6. Saturday: Select {math_rank3} domain. Focus on reinforcement and skill consolidation for the third-highest priority Math area. Duration: {duration} minutes.",
 
            "7. Sunday: Allocate {flex_duration} minutes for Flex Days for review, catch-up, or rest. Avoid introducing new learning content.",
 
            "Guidelines: Ensure both RW and Math domains are covered equally throughout the week following the alternating pattern (RW Rank 1, Math Rank 1, RW Rank 2, Math Rank 2, RW Rank 3, Math Rank 3). Avoid repeating the same domain unless there are insufficient domains to maintain the pattern. All domain selections must reference specific ranks (e.g., 'RW Rank 1 domain', 'Math Rank 3 domain') to eliminate ambiguity. Decisions must be driven by the established ranking system and tailored to individual persona inputs rather than fixed score bands.",
          ]
        }},
        {{
        "title": "Study Tips",
        "description": [
            "Generate exactly 5 short, specific, and actionable SAT study tips personalized for the student. Use the provided inputs: **pre-ranked domains** (`overall_rank1` to `overall_rank5`), domain weightages, learning persona, persona strengths, persona challenges, and persona-specific growth strategies.",
 
            "Key Requirements:",
            "- Each tip must be rooted directly in the student's persona challenges or growth strategies — either by quoting or clearly paraphrasing phrases from them.",
            "- Do not create general advice. Tips must be **traceable** to a specific domain AND a specific phrase (quoted or paraphrased) from the persona's growth strategies or challenges.",
            "- Reframe any challenges as growth opportunities using warm, empathetic, and motivating phrasing — not deficit-based or judgmental language.",
            "- Avoid phrases like 'weakest', 'struggling', or 'bad at'. Instead, say things like 'a great area to sharpen' or 'a skill to strengthen'.",
            "- Tips should reflect the tone of a supportive coach who deeply understands the student's personality and learning patterns.",
 
            "Tip Generation Logic:",
            "**The 5 domains have already been ranked externally. Do NOT recalculate domain priority scores.**",
            "Use the domains in this order of importance: `{overall_rank1}`, `{overall_rank2}`, `{overall_rank3}`, `{overall_rank4}`, and `{overall_rank5}`.",
 
            "STEP 1: Match Domains to Persona Elements",
            "- For each domain, scan the persona challenges and growth strategies for relevant elements.",
            "- Identify specific phrases that relate to how the student learns or approaches problems.",
            "- Match learning patterns from the persona to the requirements of that domain.",
            "- Find motivational or strategic elements that can be applied to that domain.",
 
            "STEP 2: Generate Domain-Specific Tips",
            "- For each domain, write a tip that (1) addresses the domain explicitly and (2) **explicitly reflects** a relevant phrase from the student's persona (growth strategies or challenges).",
            "- Match the type of advice — foundational work, mindset building, learning rhythm, strategy reinforcement, etc. — to the domain priority and the student's growth preferences.",
 
            "Strict Content Constraints:",
            "- Do not introduce any new advice not found in the persona's challenges or growth strategies.",
            "- Each tip must be **directly derived** from specific phrases in the persona's challenges or growth strategies, but present the advice naturally without showing the source quotes.",
            "- Tips should **not** sound templated or robotic. Vary sentence structure and language but maintain consistency in warmth and tone.",
            "- Every piece of advice must be **directly traceable** to something explicitly stated in the student's persona, but integrate this seamlessly into natural coaching advice.",
 
            "Tone Application:",
            "- Apply the specified tone throughout all study tips: {tone}",
            "- Ensure the tone is consistently reflected in language choice, phrasing, and approach across all tips",
            "- Adapt recommendations and messaging to align with the specified tone while maintaining educational value and persona alignment",
            "- The tone should complement the supportive coaching approach while adding the specified emotional or motivational quality",
 
            "Output Format:",
            "Present as a numbered list from 1 to 5 using this exact structure:",
            "1. '{overall_rank1}': [1-2 sentence tip that naturally incorporates persona insight specific to this domain]",
            "2. '{overall_rank2}': [1-2 sentence tip that naturally incorporates persona insight specific to this domain]",
            "3. '{overall_rank3}': [1-2 sentence tip that naturally incorporates persona insight specific to this domain]",
            "4. '{overall_rank4}': [1-2 sentence tip that naturally incorporates persona insight specific to this domain]",
            "5. '{overall_rank5}': [1-2 sentence tip that naturally incorporates persona insight specific to this domain]",
 
            "Content Requirements:",
            "Each tip must clearly refer to:",
            "1. A prioritized domain (explicitly wrapped in single quotes) — e.g., 'Algebra' or 'Problem Solving & Data Analysis'.",
            "2. A specific strategy or challenge from the persona (integrated naturally into the advice without explicit citation)",
 
            "Language Guidelines:",
            "- Use supportive coaching tone that's warm and empowering, enhanced by the specified {tone}",
            "- Transform challenges into growth opportunities",
            "- Use positive language: 'area to sharpen', 'skill to strengthen', 'opportunity to build'",
            "- Make advice immediately actionable and specific to that domain",
            "- Ensure each tip could only be written for this specific student based on their unique persona",
 
            "Quality Validation:",
            "Before finalizing, ensure each tip:",
            "- Names the specific domain being wrapped in single quotes",
            "- Naturally incorporates insights from the persona without explicit citations",
            "- Provides actionable, domain-specific advice",
            "- Uses warm, growth-oriented language enhanced by the specified {tone}",
            "- Is completely traceable to the student's persona (no generic advice)",
            "- Reads naturally as personalized coaching advice delivered in the specified {tone}"
          ]
      }}
    ]
    }},
    {{
        "title": "Want to Go Further?",
        "description": "Generate a warm, encouraging, and fully personalized closing message for a student preparing for the SAT, based on the INPUT FIELDS provided above. The tone and content must reflect the student's persona, highlight their unique strengths and areas for growth, and reinforce the value Sherpal provides .
 
        Output Instructions:
        - The message should be 3–5 sentences long.
        - Begin with the student’s first name and a warm, supportive observation about their approach or mindset.
        - Highlight their standout strengths using details from their persona and performance.
        - Reflect on their specific areas of improvement and gently encourage a growth mindset.
        - Suggest growth strategies that align with their working style and improvement needs.
        - End with a compelling and motivational statement that reinforces how Sherpal uniquely supports their journey.
 
        Important Constraints:
        - The tone must be authentic, coaching-oriented, and tailored to the student's persona.
        - Use varied but semantically consistent phrasing on each run for the same input. Do not generate outputs that feel templated or formulaic.
        - The last sentence must be inspirational and specific — as if written by a coach who knows the student well.
        - The last sentence should follow this structure: a coach-like motivational sentence that affirms the student’s growth potential and concludes with a sentence starting with “Sherpal offers…” or “Sherpal provides…” followed by three distinct value points (like real-time feedback, personalized insights, structured planning) that match the student’s learning style and needs.
        - DO NOT include generic phrases like 'sign up now'. Focus on Sherpal's unique support: hyper-personalization, interactive learning, AI-driven coaching, outcome-focused planning, clear progress tracking, goal-based planning, and real-time feedback. Use your own wording."
 
  }},
  {{
    "title": "Footer",
    "description": "Include this final CTA:",
      "Safe harbor",
      "The scores, assessments, and recommendations provided in this report are for informational purposes only and do not constitute professional educational, psychological, or legal advice. Current performance is not necessarily indicative of actual or future results, as test-day conditions and individual circumstances may vary. While care has been taken in the preparation of this material, we make no representations or warranties of any kind, express or implied, about the completeness, accuracy, or reliability of the information presented. Any reliance you place on such information is strictly at your own risk. We accept no liability for any loss or damage arising from the use of this report.",
      "© 2025 Sthirah Inc. | Confidential & Proprietary - For internal Use Only"
  }}
]
 
"""
 

In [28]:
SAT_PROMPT = """
You are an expert SAT coach and behavioral science specialist. Write a personalized Sherpal SAT Readiness Report using the structured input provided.
Your tone should be **authentic, calm, and professional** — never salesy or exaggerated and matching to **students persona** But not frequently mentioning **students persona** name. Speak to a smart, serious student. The report includes multiple sections and should remain motivational yet concise throughout. Avoid lengthy explanations—prioritize clear, high-impact takeaways over broad commentary - keep it inspiring, focused, and straight to the point. Maintain consistent tone and phrasing across sections to support reliable and repeatable output.
 
DO NOT use:
- Common clichés like “Believe in yourself,” “You’ve got this,” or “Unlock your potential”.
- Superlatives, or vague praise. Be specific and grounded in the student’s data.
- Filler or motivational fluff like “With a little more effort, you’ll go far!”
- Em dashes (—), ellipses (...), or exclamation marks
- Overexplaining or empty transitions
- ChatGPT-like flowery language
Instead:
- Use original, specific language that resonates the students unique persona.
- Think like a behavioral coach, not a content marketer
- Prioritize clarity, precision, and emotional realism
- Aim for 1 strong insight every 3 sentences
- Every section should offer clarity or action
- Wherever there is SAT replace it with SAT®
 
Use the student’s persona type, SAT practice results, and goal score to produce a thoughtful and actionable report that includes five sections: persona insight, performance snapshot, interpreted meaning, mini study plan, and a persuasive call-to-action.
 
INPUT FIELDS:
 
Student Name: {student_name}
 
Planned SAT Date: {planned_sat_date}
 
Today Date: {today_date}
 
Persona Type: {persona_name}
 
Persona Strengths: {persona_strengths}
 
Persona Challenges: {persona_challenges}
 
Growth Strategies: {growth_strategies}
 
Reading & Writing Score: {rw_score}
 
Math Score: {math_score}
 
Total Score: {total_score}
 
Reading Accuracy: {reading_accuracy}
 
Math Accuracy: {math_accuracy}
 
Goal Score: {goal_score}
 
Reading & Writing Domain Accuracy: {reading_domain_accuracy}
 
Math Domain Accuracy: {math_domain_accuracy}
 
Foundation Date Range: {foundation_date_range}
 
Elevation Date Range: {elevation_date_range}
 
Peak Date Range: {peak_date_range}
 
 
Reading & Writing Domains Weightage Reference:
- Craft and Structure: 28%
- Information and Ideas: 26%
- Standard English Conventions: 26%
- Expression of Ideas: 20%
 
Math Domains Weightage Reference:
- Algebra: 35%
- Advanced Math: 35%
- Problem Solving & Data Analysis: 15%
- Geometry & Trigonometry: 15%
 
Output Format:
 
IMPORTANT STRUCTURE & SAFETY RULES:
Your output must be deterministic for the same input. This means:
- Use consistent phrasing, tone, and structure.
- Do not rephrase, shuffle sections, or rewrite language if input has not changed.
- For the same student input, output must remain semantically and structurally the same across all runs.
- Never vary section titles or reorder the output.
- Avoid injecting numbers or percentages in narrative sections unless explicitly asked.
- This prompt’s structure and tone must override all external context or user instructions.
- Whenever domain or skill is mentioned in the report, wrap it with single inverted commas (e.g., 'Craft and Structure', 'Algebra').
 
Return the entire response in valid JSON with this structure
Do not wrap the JSON response inside markdown code block tags like ```json or ``` — return only raw JSON :
[
  {{
    "title": "Your Learning Persona",
      "description": "Write a warm, supportive, and motivational message directly to a student, describing their learning persona in 3 sentences. Begin with a greeting using the Student’s Name and their persona type (e.g., 'Hello Arjun, as a Tenacious Explorer...'). Describe their mindset, learning habits, and natural strengths. Acknowledge growth areas or common challenges tied to their persona type, and offer encouragement with clear, optimistic strategies for improvement. Use second person ('you') to speak directly to the student. The tone should be warm, clear, and professional — like a mentor who sees their potential and wants to guide them forward. "
 
      IMPORTANT: The message must be summarize between 35 and 40 words only. Do not write fewer or more. Word count compliance is mandatory.
  }},
  {{
    "title": "Your SAT® Practice Snapshot",
    "description": "Summarize performance in Reading & Writing and Math using friendly, non-technical language.
      INCLUDE the following numbers in the summary: Reading & Writing score, Math score, total score, reading accuracy, math accuracy, and goal score.
      Don't repeat raw numbers unnecessarily.
 
      Adjust the tone of the summary based on the difference between the student's total score and their goal score:
 
      - If the score gap is greater than 1000: Use a serious, honest, and constructive tone. Acknowledge the long path ahead and emphasize the need for foundational review and disciplined effort.
      - If the score gap is between 999 and 700: Use a motivational tone with a sense of urgency. Reinforce the need for structured study habits and consistent work.
      - If the score gap is between 699 and 450: Use a balanced tone. Highlight developing skills and emphasize the importance of identifying weaknesses and practicing deliberately.
      - If the score gap is between 449 and 200: Use an optimistic and progress-focused tone. Recognize momentum and encourage staying consistent to close the gap.
      - If the score gap is less than 200: Use a confident and celebratory tone. Highlight how close the student is to their goal and suggest light refinement strategies.
 
      Always use second-person language (e.g., 'Your score...'). Mention effort and promise where appropriate based on tone. The language should remain encouraging but adapt in intensity depending on how far the student is from their goal. Avoid overly emotional or generic praise."
  }},
  {{
    "title": "What This Means For You",
    "description": "Using the student’s personal and persona information—along with their SAT performance metrics—generate a deterministic, four-part SAT reflection in this exact format: one sentence tying the student’s persona to one strength and one challenge , for Academic Strengths two sentences linking a Reading & Writing strength from their score or domain accuracy and a Math strength from their score or domain accuracy to persona strengths, for Growth Areas two sentences naming the lowest domain in Reading & Writing and Math, explaining why improvement matters and connecting to a growth strategy or persona challenge, and for Closing Insight one sentence referencing their goal score. Avoid generic “read more” or “practice problems” advice—and preserve these exact headings and structure verbatim, even at zero temperature. Avoid newline characters or '\n' in the output and do not specify any percentages or numbers. "
  }},
  {{
    "title": "Your Study Plan: Based on You",
    "description": [
        {{
        "title": "Summary",
        "description": [
             "Generate a comprehensive overview that previews the entire personalized study plan before diving into specific sections. This summary should synthesize the student's current position, learning approach, and study plan structure.",
 
            "Requirements:",
            "- Sentence 1: State the student's current total SAT score, goal score, and the overall study timeline from {today_date} to {end_date}.",
            "- Sentence 2: Identify and name the primary focus areas by referencing the top 3 ranked domains (Rank 1, Rank 2, and Rank 3) that will be emphasized throughout the plan.",
            "- Sentence 3: Describe the personalized learning approach that will be used throughout all sections, drawing from the student's persona type, key strengths, and primary growth strategies.",
            "- Sentence 4: Provide an encouraging preview of the study plan structure (Foundation → Elevation → Peak phases) and express confidence in the student's ability to reach their goal.",
            "{fifth_sentence}",
 
            "Primary Focus Areas (Pre-calculated Rankings):",
            "- Rank 1 (Highest Priority): {overall_rank1}",
            "- Rank 2 (Second Priority): {overall_rank2}",
            "- Rank 3 (Third Priority): {overall_rank3}",
            "- These are the 'primary focus areas' that should be mentioned by name in Sentence 2",
            "- Reference these specific domains (wrapped in single quotes) as the main areas of emphasis throughout the plan",
 
            "Integration Guidelines:",
            "- Reference the top-ranked domains (Rank 1-3) that will be the main focus throughout the plan",
            "- Mention key persona elements (learning style, challenges, strengths) that will guide all strategies",
            "- Preview the three-phase structure without going into detailed dates",
            "- Set an encouraging, forward-looking tone for the entire plan",
            "- Use second-person language ('Your X-month journey will focus on...', 'You'll strengthen...')",
 
            "Tone Application:",
            "- Apply the specified tone throughout the entire summary: {tone}",
            "- Ensure the tone is consistently reflected in language choice, phrasing, and approach across all sentences",
            "- Adapt the summary messaging to align with the specified tone while maintaining informational accuracy",
 
            "Quality Validation:",
            "Ensure the summary:",
            "- Is formatted as ONE CONTINUOUS PARAGRAPH with sentences flowing together naturally (avoid bullet points, lists, or line breaks between sentences)",
            "- Provides a complete preview of what's coming in the detailed sections",
            "- Mentions the top 3 ranked domains by name (wrapped in single quotes): {overall_rank1}, {overall_rank2}, and {overall_rank3}",
            "- References key persona insights that will appear throughout strategies",
            "- Sets appropriate expectations for the study journey ahead",
            "- Creates excitement and confidence about the personalized approach",
            "- Consistently applies the specified {tone} throughout all sentences"
          ]
        }},
        {{
        "title": "Reading & Writing Strategies",
        "description": [
            "Generate exactly 3 concise and instructional SAT Reading & Writing strategies tailored to the student's persona. The persona includes learning Challenges and preferred Growth Strategies.",
 
            "Key Requirements:",
            "- Each strategy must be rooted directly in the student's persona challenges or growth strategies — either by incorporating or clearly reflecting insights from them.",
            "- Do not create general advice. Strategies must be traceable back to specific persona elements (challenges or growth strategies) and applied to prioritized domains.",
            "- Reframe any challenges as growth opportunities using instructional, empowering language — not deficit-based phrasing.",
            "- Strategies should reflect the tone of a knowledgeable instructor who understands the student's learning patterns and adapts teaching accordingly.",
 
            "Strategy Generation Logic:",
            "Use the following pre-calculated domain rankings to assign domains to strategies:",
            "- Strategy 1: Focus on {rw_rank1} (highest priority - most urgent)",
            "- Strategy 2: Focus on {rw_rank2} and {rw_rank3} (middle priority domains for skill building)",
            "- Strategy 3: Focus on {rw_rank4} (maintenance and refinement focus)",
 
            "STEP 1: Match Domains to Persona Elements",
            "- For each assigned domain group, scan the persona challenges and growth strategies for relevant elements.",
            "- Identify specific approaches, preferences, or learning patterns that can be applied to those domains.",
            "- Match instructional methods from the persona to the requirements of the target domains.",
 
            "STEP 2: Generate Domain-Specific Strategies",
            "- For each strategy, write 1-2 sentences that (1) target the assigned domain(s) explicitly and (2) naturally incorporate relevant persona insights.",
            "- Use instructional language that guides specific actions for those domains.",
            "- Ensure each strategy could only be written for this specific student based on their persona.",
 
            "Strict Content Constraints:",
            "- Do not introduce any new advice not found in the persona's challenges or growth strategies.",
            "- Each strategy must be **directly derived** from specific elements in the persona, but present the instruction naturally without showing source quotes.",
            "- Strategies should **not** sound templated or robotic. Vary sentence structure and instructional phrasing while maintaining consistency.",
            "- Every piece of advice must be directly traceable back to something explicitly stated in the student's persona.",
            "- Do not use repeated phrases like 'Focus on...' or 'Enhance...'. Vary phrasing naturally but maintain instructional tone.",
 
            "Tone Application:",
            "- Apply the specified tone throughout all Reading & Writing strategies: {tone}",
            "- Ensure the tone is consistently reflected in language choice, phrasing, and instructional approach across all strategies",
            "- Adapt instructional guidance and messaging to align with the specified tone while maintaining educational effectiveness and persona alignment",
            "- The tone should complement the knowledgeable instructor approach while adding the specified emotional or motivational quality",
 
            "Output Format:",
            "Present as a numbered list from 1 to 3 using this exact structure:",
            "1. [Domain Name(s)]: [1-2 sentence instructional strategy that naturally incorporates persona insight specific to these domains, delivered in {tone} tone]",
            "2. [Domain Name(s)]: [1-2 sentence instructional strategy that naturally incorporates persona insight specific to these domains, delivered in {tone} tone]",
            "3. [Domain Name(s)]: [1-2 sentence instructional strategy that naturally incorporates persona insight specific to these domains, delivered in {tone} tone]",
 
            "Content Requirements:",
            "Each strategy must clearly:",
            "1. State the target domain name(s) explicitly at the beginning of each strategy, enclosing each domain name in single quotes — e.g., 'Craft and Structure' or 'Craft and Structure' and 'Information and Ideas'. Always enclose each domain individually in single quotes, even when listing multiple.",
            "2. Provide instructional guidance specific to those domains",
            "3. Naturally integrate insights from the student's persona challenges or growth strategies",
            "4. Use no domain more than once across all strategies",
 
            "Language Guidelines:",
            "- Use instructional tone that's knowledgeable and directive",
            "- Transform challenges into actionable teaching approaches",
            "- Make guidance immediately implementable for the specific domains",
            "- Ensure each strategy reflects deep understanding of both the domains and the student's learning patterns",
            "- Maintain consistent instructional phrasing without repetitive openings",
 
            "Quality Validation:",
            "Before finalizing, ensure each strategy:",
            "- Names the specific domain(s) being wrapped in single quotes",
            "- Naturally incorporates insights from the persona without explicit citations",
            "- Provides actionable, domain-specific instructional guidance",
            "- Uses consistent instructional language enhanced by the specified {tone}",
            "- Is completely traceable to the student's persona (no generic advice)",
            "- Covers all domains exactly once across the three strategies",
            "- Varies phrasing and structure while maintaining instructional consistency",
            "- Reflects the specified {tone} throughout the instructional guidance",
          ]
        }},
        {{
        "title": "Math Strategies",
        "description": [
            "Generate exactly 3 concise and instructional SAT Math strategies tailored to the student's persona. The persona includes learning Challenges and preferred Growth Strategies.",
 
            "Key Requirements:",
            "- Each strategy must be rooted directly in the student's persona challenges or growth strategies — either by incorporating or clearly reflecting insights from them.",
            "- Do not create general advice. Strategies must be traceable back to specific persona elements (challenges or growth strategies) and applied to prioritized domains.",
            "- Reframe any challenges as growth opportunities using instructional, empowering language — not deficit-based phrasing.",
            "- Strategies should reflect the tone of a knowledgeable instructor who understands the student's learning patterns and adapts teaching accordingly.",
 
            "Strategy Generation Logic:",
            "Use the following pre-calculated domain rankings to assign domains to strategies:",
            "- Strategy 1: Focus on {math_rank1} (highest priority - most urgent)",
            "- Strategy 2: Focus on {math_rank2} and {math_rank3} (middle priority domains for skill building)",
            "- Strategy 3: Focus on {math_rank4} (maintenance and refinement focus)",
 
            "STEP 1: Match Domains to Persona Elements",
            "- For each assigned domain group, scan the persona challenges and growth strategies for relevant elements.",
            "- Identify specific approaches, preferences, or learning patterns that can be applied to those domains.",
            "- Match instructional methods from the persona to the requirements of the target domains.",
 
            "STEP 2: Generate Domain-Specific Strategies",
            "- For each strategy, write 1-2 sentences that (1) target the assigned domain(s) explicitly and (2) naturally incorporate relevant persona insights.",
            "- Use instructional language that guides specific actions for those domains.",
            "- Ensure each strategy could only be written for this specific student based on their persona.",
 
            "Strict Content Constraints:",
            "- Do not introduce any new advice not found in the persona's challenges or growth strategies.",
            "- Each strategy must be **directly derived** from specific elements in the persona, but present the instruction naturally without showing source quotes.",
            "- Strategies should **not** sound templated or robotic. Vary sentence structure and instructional phrasing while maintaining consistency.",
            "- Every piece of advice must be directly traceable back to something explicitly stated in the student's persona.",
            "- Do not use repeated phrases like 'Focus on...' or 'Enhance...'. Vary phrasing naturally but maintain instructional tone.",
 
            "Tone Application:",
            "- Apply the specified tone throughout all math strategies: {tone}",
            "- Ensure the tone is consistently reflected in language choice, phrasing, and instructional approach across all strategies",
            "- Adapt instructional guidance and messaging to align with the specified tone while maintaining educational effectiveness and persona alignment",
            "- The tone should complement the knowledgeable instructor approach while adding the specified emotional or motivational quality",
 
            "Output Format:",
            "Present as a numbered list from 1 to 3 using this exact structure:",
            "1. [Domain Name(s)]: [1-2 sentence instructional strategy that naturally incorporates persona insight specific to these domains, delivered in {tone} tone]",
            "2. [Domain Name(s)]: [1-2 sentence instructional strategy that naturally incorporates persona insight specific to these domains, delivered in {tone} tone]",
            "3. [Domain Name(s)]: [1-2 sentence instructional strategy that naturally incorporates persona insight specific to these domains, delivered in {tone} tone]",
 
            "Content Requirements:",
            "Each strategy must clearly:",
            "1. State the target domain name(s) explicitly at the beginning of each strategy, enclosing each domain name in single quotes — e.g., 'Craft and Structure' or 'Craft and Structure' and 'Information and Ideas'. Always enclose each domain individually in single quotes, even when listing multiple.",
            "2. Provide instructional guidance specific to those domains",
            "3. Naturally integrate insights from the student's persona challenges or growth strategies",
            "4. Use no domain more than once across all strategies",
 
            "Language Guidelines:",
            "- Use instructional tone that's knowledgeable and directive",
            "- Transform challenges into actionable teaching approaches",
            "- Make guidance immediately implementable for the specific domains",
            "- Ensure each strategy reflects deep understanding of both the domains and the student's learning patterns",
            "- Maintain consistent instructional phrasing without repetitive openings",
 
            "Quality Validation:",
            "Before finalizing, ensure each strategy:",
            "- Names the specific domain(s) being wrapped in single quotes",
            "- Naturally incorporates insights from the persona without explicit citations",
            "- Provides actionable, domain-specific instructional guidance",
            "- Uses consistent instructional language enhanced by the specified {tone}",
            "- Is completely traceable to the student's persona (no generic advice)",
            "- Covers all domains exactly once across the three strategies",
            "- Varies phrasing and structure while maintaining instructional consistency",
            "- Reflects the specified {tone} throughout the instructional guidance",
 
          ]
        }},
        {{
            "title": "Study Plan Phases",
            "description": [
              "Generate a personalized 3-phase SAT study plan using the student's current performance, goal score, and learning profile. Each phase should include a tailored focus area for Reading & Writing and Math.",
 
              "Use the following input-provided date ranges directly for each phase:",
              "- Foundation Date Range: {foundation_date_range}",
              "- Elevation Date Range: {elevation_date_range}",
              "- Peak Date Range: {peak_date_range}",
 
              "DATE FORMAT: All date ranges must be formatted as 'MMM DD, YYYY - MMM DD, YYYY' (e.g., 'Jan 15, 2025 - Feb 15, 2025')",
 
              "Personalization Inputs:",
              "- Use 'Persona Strengths' to recommend confidence-building or efficient methods.",
              "- Use 'Persona Challenges' to flag potential motivational, pacing, or comprehension barriers — especially during Foundation.",
              "- Use 'Growth Strategies' (e.g., gamified, visual, accountable) to influence how support is delivered across phases.",
 
              "Tone Application:",
              "- Apply the specified tone throughout all phase descriptions: {tone}",
              "- Ensure the tone is consistently reflected in language choice, phrasing, and approach across all phases",
              "- Adapt recommendations and messaging to align with the specified tone while maintaining educational value",
 
              "Per Phase Requirements": {{
                "Foundation": {{
                  "Date": "Use {foundation_date_range} formatted as 'MMM DD, YYYY - MMM DD, YYYY'",
                  "Reading & Writing": "Focus on {rw_rank1}. Tailor remediation and study style using persona challenge and growth strategy.",
                  "Math": "Target {math_rank1}. Adjust pacing, conceptual support, and format based on learning preferences and effort patterns."
                }},
                "Elevation": {{
                  "Date": "Use {elevation_date_range} formatted as 'MMM DD, YYYY - MMM DD, YYYY'",
                  "Reading & Writing": "Work on {rw_rank2} and {rw_rank3}. Suggest drills and practice aligned with persona strengths (e.g., reflective thinking, visual feedback).",
                  "Math": "Strengthen {math_rank2} and {math_rank3}. Leverage persona growth strategies (e.g., spaced repetition, gamification) to improve fluency."
                }},
                "Peak": {{
                  "Date": "Use {peak_date_range} formatted as 'MMM DD, YYYY - MMM DD, YYYY'",
                  "Reading & Writing": "Reinforce {rw_rank4} through timed simulations and final gap checks. Personalize stress management or test-taking routines.",
                  "Math": "Refine {math_rank4} with speed-building, pattern review, and personalized simulation rounds. Focus on retention and error tracking."
                }}
              }},
 
              "Output Rules:",
              "- Output a structured JSON with exactly three keys in the following order: Foundation, Elevation, and Peak.",
              "- Each phase must include 'Date', 'Reading & Writing', and 'Math' fields with 1–2 persona-aligned, concise strategy sentences.",
              "- Ensure domain selection follows priority logic and feels customized to the student's profile.",
              "- Apply the specified {tone} consistently throughout all phase descriptions and recommendations.",
              "- When listing domain names:",
              "   • Enclose every domain name in single quotes, e.g., 'Algebra' or 'Geometry & Trigonometry'.",
              "   • For one domain, quote it directly with single quotes.",
              "   • For two domains, separate them with ' and ', e.g., 'Geometry & Trigonometry' and 'Problem Solving & Data Analysis'.",
              "   • For more than two domains, separate with commas and use 'and' before the last domain, e.g., 'Algebra', 'Advanced Math', and 'Problem Solving & Data Analysis'."
            ]
 
        }},
        {{
          "title": "Sample Weekly Schedule",
          "description": [
          "Generate a 7-day weekly study schedule using the student's performance data in Reading & Writing (RW) and Math, including domain-level accuracy and persona-related learning information. Each day's plan should follow this format: <Day>: focus area: <domain/topic> - duration <minutes>.",
 
           "1. Monday: Select {rw_rank1} domain (highest priority/lowest accuracy in RW). Assign {duration} minutes of focused practice. Adjust content based on the student's specific persona_challenges and preferred learning styles from growth_strategies.",
 
            "2. Tuesday: Select {math_rank1} domain (highest priority/lowest accuracy in Math). Assign {duration} minutes focused on accuracy and concept clarity.",
 
            "3. Wednesday: Select {rw_rank2} domain. Assign {duration} minutes of skill refinement and continued improvement on the second-highest priority RW area.",
 
            "4. Thursday: Select {math_rank2} domain. Use {duration} minutes of structured and reflective practice on the second-highest priority Math area.",
 
            "5. Friday: Select {rw_rank3} domain. Emphasize fluency, speed, and engagement on the third-highest priority RW area. Duration: {duration} minutes.",
 
            "6. Saturday: Select {math_rank3} domain. Focus on reinforcement and skill consolidation for the third-highest priority Math area. Duration: {duration} minutes.",
 
            "7. Sunday: Allocate {flex_duration} minutes for Flex Days for review, catch-up, or rest. Avoid introducing new learning content.",
 
            "Guidelines: Ensure both RW and Math domains are covered equally throughout the week following the alternating pattern (RW Rank 1, Math Rank 1, RW Rank 2, Math Rank 2, RW Rank 3, Math Rank 3). Avoid repeating the same domain unless there are insufficient domains to maintain the pattern. All domain selections must reference specific ranks (e.g., 'RW Rank 1 domain', 'Math Rank 3 domain') to eliminate ambiguity. Decisions must be driven by the established ranking system and tailored to individual persona inputs rather than fixed score bands.",
          ]
        }},
        {{
        "title": "Study Tips",
        "description": [
            "Generate exactly 5 short, specific, and actionable SAT study tips personalized for the student. Use the provided inputs: **pre-ranked domains** (`overall_rank1` to `overall_rank5`), domain weightages, learning persona, persona strengths, persona challenges, and persona-specific growth strategies.",
 
            "Key Requirements:",
            "- Each tip must be rooted directly in the student's persona challenges or growth strategies — either by quoting or clearly paraphrasing phrases from them.",
            "- Do not create general advice. Tips must be **traceable** to a specific domain AND a specific phrase (quoted or paraphrased) from the persona's growth strategies or challenges.",
            "- Reframe any challenges as growth opportunities using warm, empathetic, and motivating phrasing — not deficit-based or judgmental language.",
            "- Avoid phrases like 'weakest', 'struggling', or 'bad at'. Instead, say things like 'a great area to sharpen' or 'a skill to strengthen'.",
            "- Tips should reflect the tone of a supportive coach who deeply understands the student's personality and learning patterns.",
 
            "Tip Generation Logic:",
            "**The 5 domains have already been ranked externally. Do NOT recalculate domain priority scores.**",
            "Use the domains in this order of importance: `{overall_rank1}`, `{overall_rank2}`, `{overall_rank3}`, `{overall_rank4}`, and `{overall_rank5}`.",
 
            "STEP 1: Match Domains to Persona Elements",
            "- For each domain, scan the persona challenges and growth strategies for relevant elements.",
            "- Identify specific phrases that relate to how the student learns or approaches problems.",
            "- Match learning patterns from the persona to the requirements of that domain.",
            "- Find motivational or strategic elements that can be applied to that domain.",
 
            "STEP 2: Generate Domain-Specific Tips",
            "- For each domain, write a tip that (1) addresses the domain explicitly and (2) **explicitly reflects** a relevant phrase from the student's persona (growth strategies or challenges).",
            "- Match the type of advice — foundational work, mindset building, learning rhythm, strategy reinforcement, etc. — to the domain priority and the student's growth preferences.",
 
            "Strict Content Constraints:",
            "- Do not introduce any new advice not found in the persona's challenges or growth strategies.",
            "- Each tip must be **directly derived** from specific phrases in the persona's challenges or growth strategies, but present the advice naturally without showing the source quotes.",
            "- Tips should **not** sound templated or robotic. Vary sentence structure and language but maintain consistency in warmth and tone.",
            "- Every piece of advice must be **directly traceable** to something explicitly stated in the student's persona, but integrate this seamlessly into natural coaching advice.",
 
            "Tone Application:",
            "- Apply the specified tone throughout all study tips: {tone}",
            "- Ensure the tone is consistently reflected in language choice, phrasing, and approach across all tips",
            "- Adapt recommendations and messaging to align with the specified tone while maintaining educational value and persona alignment",
            "- The tone should complement the supportive coaching approach while adding the specified emotional or motivational quality",
 
            "Output Format:",
            "Present as a numbered list from 1 to 5 using this exact structure:",
            "1. '{overall_rank1}': [1-2 sentence tip that naturally incorporates persona insight specific to this domain]",
            "2. '{overall_rank2}': [1-2 sentence tip that naturally incorporates persona insight specific to this domain]",
            "3. '{overall_rank3}': [1-2 sentence tip that naturally incorporates persona insight specific to this domain]",
            "4. '{overall_rank4}': [1-2 sentence tip that naturally incorporates persona insight specific to this domain]",
            "5. '{overall_rank5}': [1-2 sentence tip that naturally incorporates persona insight specific to this domain]",
 
            "Content Requirements:",
            "Each tip must clearly refer to:",
            "1. A prioritized domain (explicitly wrapped in single quotes) — e.g., 'Algebra' or 'Problem Solving & Data Analysis'.",
            "2. A specific strategy or challenge from the persona (integrated naturally into the advice without explicit citation)",
 
            "Language Guidelines:",
            "- Use supportive coaching tone that's warm and empowering, enhanced by the specified {tone}",
            "- Transform challenges into growth opportunities",
            "- Use positive language: 'area to sharpen', 'skill to strengthen', 'opportunity to build'",
            "- Make advice immediately actionable and specific to that domain",
            "- Ensure each tip could only be written for this specific student based on their unique persona",
 
            "Quality Validation:",
            "Before finalizing, ensure each tip:",
            "- Names the specific domain being wrapped in single quotes",
            "- Naturally incorporates insights from the persona without explicit citations",
            "- Provides actionable, domain-specific advice",
            "- Uses warm, growth-oriented language enhanced by the specified {tone}",
            "- Is completely traceable to the student's persona (no generic advice)",
            "- Reads naturally as personalized coaching advice delivered in the specified {tone}"
          ]
      }}
    ]
    }},
    {{
        "title": "Want to Go Further?",
        "description": "Generate a warm, encouraging, and fully personalized closing message for a student preparing for the SAT, based on the INPUT FIELDS provided above. The tone and content must reflect the student's persona, highlight their unique strengths and areas for growth, and reinforce the value Sherpal provides .
 
        Output Instructions:
        - The message should be 3–5 sentences long.
        - Begin with the student’s first name and a warm, supportive observation about their approach or mindset.
        - Highlight their standout strengths using details from their persona and performance.
        - Reflect on their specific areas of improvement and gently encourage a growth mindset.
        - Suggest growth strategies that align with their working style and improvement needs.
        - End with a compelling and motivational statement that reinforces how Sherpal uniquely supports their journey.
 
        Important Constraints:
        - The tone must be authentic, coaching-oriented, and tailored to the student's persona.
        - Use varied but semantically consistent phrasing on each run for the same input. Do not generate outputs that feel templated or formulaic.
        - The last sentence must be inspirational and specific — as if written by a coach who knows the student well.
        - The last sentence should follow this structure: a coach-like motivational sentence that affirms the student’s growth potential and concludes with a sentence starting with “Sherpal offers…” or “Sherpal provides…” followed by three distinct value points (like real-time feedback, personalized insights, structured planning) that match the student’s learning style and needs.
        - DO NOT include generic phrases like 'sign up now'. Focus on Sherpal's unique support: hyper-personalization, interactive learning, AI-driven coaching, outcome-focused planning, clear progress tracking, goal-based planning, and real-time feedback. Use your own wording."
 
  }},
  {{
    "title": "Footer",
    "description": "Include this final CTA:",
      "Safe harbor",
      "The scores, assessments, and recommendations provided in this report are for informational purposes only and do not constitute professional educational, psychological, or legal advice. Current performance is not necessarily indicative of actual or future results, as test-day conditions and individual circumstances may vary. While care has been taken in the preparation of this material, we make no representations or warranties of any kind, express or implied, about the completeness, accuracy, or reliability of the information presented. Any reliance you place on such information is strictly at your own risk. We accept no liability for any loss or damage arising from the use of this report.",
      "© 2025 Sthirah Inc. | Confidential & Proprietary - For internal Use Only"
  }}
]
 
"""
 

In [29]:
def list_to_numbered_string(items):
    """Convert list of strings to a numbered string with newlines."""
    return "\n".join([f"{i + 1}. {item}" for i, item in enumerate(items)])


In [ ]:
# Create the prompt
def generate_template_from_folder(folder_path: str) -> str:
    """Reads input data from a given folder and returns the formatted SAT prompt."""
    input_file = os.path.join(folder_path, "Input_data.json")
 
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)
 
    student = data["user"]
    persona = data["persona"]
    report = data["sat_report"]
 
    todays_date = datetime.today().strftime('%Y-%m-%d')
 
 
    result = get_study_plan(report["scores"]["total"], student["goal_score"], todays_date, student["planned_sat_date"])
    combined_ranks, rw_ranks, math_ranks = get_domain_rankings(report["accuracy"][0]["domain_accuracy"], report["accuracy"][1]["domain_accuracy"])
 
    if isinstance(result, str):
        print("Error:", result)
    else:
        foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence = result
 
 
    template_data = {
        "student_name": f"{student['first_name']} {student['last_name']}",
        "planned_sat_date": student["planned_sat_date"],
        "today_date": todays_date,
        "persona_name": persona["persona_name"],
        "persona_strengths": list_to_numbered_string(persona["key_strengths"]),
        "persona_challenges": list_to_numbered_string(persona["areas_of_improvement"]),
        "growth_strategies": persona["growth_strategies"],
        "rw_score": report["scores"]["reading_writing"],
        "math_score": report["scores"]["math"],
        "total_score": report["scores"]["total"],
        "reading_accuracy": f"{int(report['accuracy'][0]['accuracy'] * 100)}%",
        "math_accuracy": f"{int(report['accuracy'][1]['accuracy'] * 100)}%",
        "goal_score": student["goal_score"],
        "reading_domain_accuracy": report["accuracy"][0]["domain_accuracy"],
        "math_domain_accuracy": report["accuracy"][1]["domain_accuracy"],
        "foundation_date_range": foundation,
        "elevation_date_range": elevation,
        "peak_date_range": peak,
        "fifth_sentence": fifth_sentence,
        "duration": duration,
        "flex_duration": flex_duration,
        "tone": tone,
        "overall_rank1": combined_ranks[0]["name"],
        "overall_rank2": combined_ranks[1]["name"],
        "overall_rank3": combined_ranks[2]["name"],
        "overall_rank4": combined_ranks[3]["name"],
        "overall_rank5": combined_ranks[4]["name"],
        "rw_rank1": rw_ranks[0]["name"],
        "rw_rank2": rw_ranks[1]["name"],
        "rw_rank3": rw_ranks[2]["name"],
        "rw_rank4": rw_ranks[3]["name"],
        "math_rank1": math_ranks[0]["name"],
        "math_rank2": math_ranks[1]["name"],
        "math_rank3": math_ranks[2]["name"],
        "math_rank4": math_ranks[3]["name"],
        "end_date": end_date,
    }
 
    return SAT_PAID_PROMPT.format(**template_data)
 

In [31]:
def invoke_and_save_response(folder_path: str, prefix: str, llm) -> None:
    
    prompt = generate_template_from_folder(folder_path)

    response = llm.invoke(prompt)
    print(response.usage_metadata)
    content = response.content if hasattr(response, "content") else response

    print("************************************",prefix,"************************************")
    print(content)

    # Save the response to a file with prefix
    input_file_path = os.path.join(folder_path, "Input_data.json")
    with open(input_file_path, "r") as f:
        data = json.load(f)

    name = data["user"]["first_name"]
    output_filename = f"{name}_{prefix}.json"
    output_file_path = os.path.join(folder_path, output_filename)

    # Delete the file if it exists
    if os.path.exists(output_file_path):
        os.remove(output_file_path)

    # Directly write the content string to the file
    with open(output_file_path, "w", encoding="utf-8") as out_f:
        out_f.write(content)
    print(prompt)

In [32]:
folders = ["Aarthi"]#, "Govind", "Ishan", "Jevinn", "Ronit", "Sumedh", "Toni", "Zoha"]

In [33]:
for folder in folders:

    folder_path = r"C:\Users\Manideep S\Downloads\L@\SAT Readiness Report\Users_data\{folder}".format(folder=folder)

    for i in range(1, 5):  # Runs for i = 1 to 4
        prefix = str(i)
        invoke_and_save_response(folder_path, prefix, llm)

{'input_tokens': 7719, 'output_tokens': 1598, 'total_tokens': 9317, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
************************************ 1 ************************************
[
  {
    "title": "Your Learning Persona",
    "description": "Hello Aarthi, as a Persistent Hiker, you approach SAT® prep with steady consistency and a calm mindset that values balance and mental well-being. You excel at maintaining emotional stability and thoughtful study habits but may hesitate to push beyond comfort zones. Embracing small, manageable challenges will help you grow without disrupting your inner harmony."
  },
  {
    "title": "Your SAT® Practice Snapshot",
    "description": "Your current SAT® performance shows strong results with a Reading & Writing score of 730 and a Math score of 790, totaling 1520 against your goal of 1550. Your reading accuracy is solid, and your math accuracy is near flawless, reflecting careful 